# Data Retrieval Process for Lake Stevens Roundabout Analysis

Erin Mee

December 9, 2025

UW MSDS - DATA 512 Human Centered Data Science Project

---

## Washington State Patrol Collision Analysis tool

![GUI for collision analysis tool](../images/collision%20analysis%20tool%20query.jpg)

https://fortress.wa.gov/wsp/collisionanalysistool/

Individual collision reports are filled out by state patrol officers. The collision location is described by a primary and secondary trafficway. Since the data comes from reports filled out by individuals there are various ways a trafficway is named, i.e. highway 9 is called *SR 9*, *SR 009*, *SR 9 NE*, etc... 

In order to query the database for all accidents at the roundabouts, the variations in naming need to be accounted for so that all accidents are included.

The collision analysis tool has a map feature I used to explore the various namings for the trafficways of interest. I queried for crashes in Lake Stevens from 1/1/2015 to 11/15/2025 to find the following variations on naming.

### State Route 9 Variations
* SR 9
* SR 9 NE
* SR9
* SR9 NE
* SR N9 NE
* STATE ROUTE 9 NE
* STATE ROAD 9
* SR-9 NE
* SR-9
* HWY 9

### State Route 204 Variations
* SR 204
* SR204
* STATE ROUTE 204
* SR-204
* HWY 204
* SR 204 NE

### Vernon Road Variations
* VERNON
* VERNON RD
* VERNON ROAD

### N Davies Road Variations
* N DAVIS RD
* NORTH DAVIS ROAD
* N. DAVIES RD
* N DAVIES ST

### Frontage Road Variations
* FRONTAGE RD

### 7th Pl NE Variations
* 7TH PL NE
* 7TH ST NE

| Trafficway pairs |  |
| --- | --- |
| SR 9 | SR 204 |
| SR 9 | N Davies Rd |
| SR 9 | Vernon Rd |
| SR 9 | 7th Pl NE |
| SR 9 | Frontage Rd |
| SR 204 | Vernon Rd |
| N Davies Rd | Frontage Rd |
| N Davies Rd | Vernon Rd |
| 900 BLK* | SR 9 NE |

### Queries used to generate the dataset

For all of these queries start date was 1/1/2015 12:00AM and end date was 11/15/2025 12:00AM. County was Snohomish and City was Lake Stevens.

Below is the code used to programmatically submit the query form for each primary and secondary roadway combination and retrieve the data.

### Imports and environment setup

In [ ]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import Select, WebDriverWait
import time
import os

### Function definition to query and retrieve the csv data file

In [2]:
def retrieve_collision_data(primary_road: str, secondary_road, file_path):
    driver = webdriver.Chrome()
    driver.get("https://fortress.wa.gov/wsp/collisionanalysistool/Query/SearchCriteria")

    # Wait for page to load
    wait = WebDriverWait(driver, 10)

    # Fill in form fields
    date_from = driver.find_element(By.ID, "ColliDateStart")
    date_from.send_keys("01/01/2015 00:00:00")

    date_to = driver.find_element(By.ID, "ColliDateEnd")
    date_to.send_keys("11/15/2025 00:00:00")

    city_select = Select(driver.find_element(By.ID, "lstCity"))
    city_select.select_by_visible_text("Lake Stevens")

    county_select = Select(driver.find_element(By.ID, "lstCounty"))
    county_select.select_by_visible_text("Snohomish")

    primary = driver.find_element(By.ID, "PrimaryTrafficway")
    primary.send_keys(primary_road)

    secondary = driver.find_element(By.ID, "SecondaryTrafficway")
    secondary.send_keys(secondary_road)

    # Submit form
    submit_button = driver.find_element(By.XPATH, "/html/body/div[2]/div[2]/form/div[2]/div[1]/div/div[1]/input")
    submit_button.click()

    # Wait for results and extract data
    time.sleep(5)

    export_button = driver.find_element(By.XPATH, '//*[@id="btnExport"]')
    data_url = export_button.get_attribute('href')

    driver.quit()

    # Download the data
    os.system(f"curl -o {file_path} '{data_url}'")

### Primary and secondary roadway combinations of naming variations

In [6]:
sr9_names = ["SR 9", "SR*9", "STATE ROUTE 9 NE", "STATE ROAD 9", "HWY 9"]
sr204_names = ["SR 204", "SR*204", "STATE ROUTE 204", "STATE ROAD 204", "HWY 204"]
sr9_crossing_roads = ["N*DAVIES*", "VERNON", "7TH*NE", "FRONTAGE"]
pairs = []
for sr9_name in sr9_names:
    for sr204_name in sr204_names:
        pairs.append((sr9_name, sr204_name))
        pairs.append((sr204_name, sr9_name))

for sr9_name in sr9_names:
    for sr9_crossing_road in sr9_crossing_roads:
        pairs.append((sr9_name, sr9_crossing_road))
        pairs.append((sr9_crossing_road, sr9_name))

for sr204_name in sr204_names:
    pairs.append((sr204_name, "VERNON"))
    pairs.append(("VERNON", sr204_name))

pairs.append(("N*DAVIES*", "FRONTAGE"))
pairs.append(("FRONTAGE", "N*DAVIES*"))
pairs.append(("N*DAVIES*", "VERNON"))
pairs.append(("VERNON", "N*DAVIES*"))

### Display resulting primary, secondary roadway naming pairs

In [8]:
column_widths = [max(len(str(item)) for item in col) for col in zip(*pairs)]

header_format = " | ".join(f"{{:<{width}}}" for width in column_widths)
print(header_format.format("Primary", "Secondary"))
print("-" * (sum(column_widths) + (len(column_widths) - 1) * 3))

for row in pairs:
    row_format = " | ".join(f"{{:<{width}}}" for width in column_widths)
    print(row_format.format(*row))

Primary          | Secondary       
-----------------------------------
SR 9             | SR 204          
SR 204           | SR 9            
SR 9             | SR*204          
SR*204           | SR 9            
SR 9             | STATE ROUTE 204 
STATE ROUTE 204  | SR 9            
SR 9             | STATE ROAD 204  
STATE ROAD 204   | SR 9            
SR 9             | HWY 204         
HWY 204          | SR 9            
SR*9             | SR 204          
SR 204           | SR*9            
SR*9             | SR*204          
SR*204           | SR*9            
SR*9             | STATE ROUTE 204 
STATE ROUTE 204  | SR*9            
SR*9             | STATE ROAD 204  
STATE ROAD 204   | SR*9            
SR*9             | HWY 204         
HWY 204          | SR*9            
STATE ROUTE 9 NE | SR 204          
SR 204           | STATE ROUTE 9 NE
STATE ROUTE 9 NE | SR*204          
SR*204           | STATE ROUTE 9 NE
STATE ROUTE 9 NE | STATE ROUTE 204 
STATE ROUTE 204  | STATE ROU

### Retrieve and save data for each roadway naming pair

Data is saved in the collision_data directory with a sequencial file name: collision_data_01.csv, collision_data_02.csv, ... etc.

*Note:* Many of the roadway naming combinations return no results but are included for coverage

In [4]:
for i, (primary, secondary) in enumerate(pairs):
    file_path = f"collision_data/collision_data_{i+1:02d}.csv"
    retrieve_collision_data(primary, secondary, file_path)

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  785k    0  785k    0     0   702k      0 --:--:--  0:00:01 --:--:--  703k
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 67083    0 67083    0     0    99k      0 --:--:-- --:--:-- --:--:--   99k
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  816k    0  816k    0     0  1227k      0 --:--:-- --:--:-- --:--:-- 1229k
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  6623    0  6623    0     0  23756      0 --:--:-- --:--:-- --:--:-- 23823
  % Total    % Received % Xferd  Average Speed   Tim

## Washington State Department of Transportation (WSDOT) Traffic Data Reporting System

![WSDOT traffic count data interface](../images/WSDOT%20traffic%20count%20GUI.jpg)

https://wsdot.public.ms2soft.com/tcds/tsearch.asp?loc=Wsdot&mod=TCDS

WSDOT records traffic data at traffic count stations along highways. A key metric that is reported is the Annual Average Daily Traffic (AADT) 

* AADT – Annual Average Daily Traffic
* DHV-30 – for Perm and WIM stations this is the traffic volume of the thirtieth highest volume
hour in the year, but for Short Count Stations this is highest volume captured
* K % – the DHV-30 as a percentage of AADT
* D % – the volume of the DHV-30 hour’s highest volume direction divided by total volume
* PA – the AADT of FHWA vehicle classes 1 through 3 (i.e., motorcycles, cars, and pickups)
* BC – the AADT of FHWA vehicles classes 4 through 13 (i.e., buses and heavy trucks)
* Src – This field will contain notes regarding the computation of these statistics if applicable. 

https://docs.ms2soft.com/docs/ms2-help-aadt-calculation